In [1]:
# N_simulation
from __future__ import division
import numpy as np
from scipy.integrate import solve_ivp
from scipy import interpolate 
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from datetime import date
import datetime
from local_functions import *
%matplotlib inline

In [2]:
# Functions for optimization (i.e producing random parametric combination)
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol

In [3]:
x1 = pd.ExcelFile('../data/Parameters.xlsx')
x2 = pd.ExcelFile('../data/Data.xlsx')

# Import parameters into df1
df1 = x1.parse('Parameters')

# assigmment of parameters to values: 
for key,val in zip(df1.Parameter,df1.Indoor_value): # keep only relvenat parameters
    exec(key + '=val')
    print(key,val)

n_reactors = 1

z = 0.085

umol_to_percent_DW 0.0014
n_days 7.0
Duration 7.0
Nint0 1.85
m0 0.15
Nintcrit 4.5
miu 0.04
Nintmax 5.5
Nintmin 0.48
Ks 14.0
Vmax 60.0
Z 0.178
KI 35.0
K0 0.5
Ka 0.05
Topt 18.0
Tmin 5.0
Tmax 35.0
n 5.0
losses20 0.0016
teta 1.047
Sopt 25.0
Smin 3.0
Smax 48.0
Shigh 30.0
Slow 12.0
KN 1.1
dNextoutdt 0.014
dNextindt 0.0
dmoutdt 0.0
Next00  4032.0
dilution 0.0


In [4]:
# Import of IMS I and T data
dfI = pd.read_csv('../data/ims_data_2020_June_I.csv',encoding= 'unicode_escape',header=None)
dfT = pd.read_csv('../data/ims_data_2020_June_T.csv',encoding= 'unicode_escape',header=None)

In [5]:
# adjust I and T data for the different experiments (datasets sizes and resolution)

# experiment 1
# get T and I data
IV1, TV1,IV1_average,TV1_average = [],[],[],[]
for vali,valT in zip(dfI.iloc[1:1010:1][4],dfT.iloc[1:1010:1][3]):
    IV1.append(float(vali))
    TV1.append(float(valT))
# calculate average T and I data per hour
for i in range(1,len(IV1),6):
    IV1_average.append(0.1*0.43*np.mean(IV1[i-1:i+6:1])) # 0.1 shading factor and 0.43 PAR factor
    TV1_average.append(np.mean(TV1[i-1:i+6:1]))
print(len(TV1_average))
print(len(IV1_average))
    
# experiment 2
# get T and I data
IV2, TV2,IV2_average,TV2_average = [],[],[],[]
for vali,valT in zip(dfI.iloc[1010:1904:1][4],dfT.iloc[1010:1904:1][3]):
    IV2.append(float(vali))
    TV2.append(float(valT))
print(len(IV2))
print(len(TV2))
# calculate average T and I data per hour
for i in range(1,len(IV2),6):
    IV2_average.append(0.1*0.43*np.mean(IV2[i-1:i+6:1])) # 0.1 shading factor and 0.43 PAR factor
    TV2_average.append(np.mean(TV2[i-1:i+6:1]))
print(len(TV2_average))
print(len(IV2_average))

168
168
894
894
149
149


In [6]:
# Lists of initial and boundary conditions and empiric results for returns 1-2:

hour2i = int(7*24)+5 # length of experiment in hours
t0_all = [0, hour2i]
tf_all = [int(7*24), hour2i + int(6*24-5)]
mi_all = [[1,1,2,2,1,1,2,2,1,1,2,2],[2,2,2,2,2,2,3,3,2,2,3,3]] # initial weights
mf_real = [[1.3,2.1,4,3,3.8,4.5,7.8,8.1,3.7,3.5,6.9,5.3],[3,2.6,5,4.6,4.4,5.7,5.7,5.8,5.1,4.7,6.55,6.9]] # final densities
S_all = [[6, 6, 6, 6, 24, 24, 24, 24, 29, 29, 29, 29],[6, 6, 40, 40, 22.5, 22.5, 22.5, 22.5, 17.5, 17.5,17.5,17.5]] # salinities
d_all = [[0,0,0,0,1,1,1,1,2,2,2,2],[0,0,0,0,1,1,1,1,0.5,0.5,0.5,0.5]] # dilution rates
Nint0_all = [[3.01,3.01,3.01,3.01,3.01,3.01,3.01,3.01,3.01,3.01,3.01,3.01],[4.09,4.09,4.09,4.09,4.09,4.09,4.09,4.09,4.09,4.09,4.09,4.09]] # initial internal N levels
Nintf_exps = [[2,3,4,5,7-1,8-1,9-1,10-1,12-2,13-2,14-2,15-2],[2,3,4,5,0,8-1,9-1,10-1,12-2,13-2,14-2,15-2]] # new numbering used to ignore samples without Nintf measurement
Nintf_real = [[3.12,2.31,2.47,2.82,2.35,2.08,1.56,1.59,2.02,1.86,1.43,1.92],[2.94,3.24,2.76,2.56,0,2.17,2.85,2.33,2.92,2.21,2.76,2.38]] # final internal N levels

TV_average = [TV1_average, TV2_average]
IV_average = [IV1_average, IV2_average]
xlabels_all = [['8 June', '11 June','15 June'],['15 June','18 June', '21 June']] #dates
l_all = [[['Brine\n6 PSU','  1:1\n24 PSU','  1:2\n29 PSU'],['Brine\n6 PSU','  1:1\n24 PSU','  1:2\n29 PSU']],[['Brine\n6 PSU','Brine+Salt\n  40 PSU','  1:1\n22.5 PSU','  1:0.5\n17.5 PSU'],['  1:1\n22.5 PSU','  1:0.5\n17.5 PSU']]] # legend labels

In [7]:
# get 9 parameters from indoor_calibration (define X as X from the indoor calibration)
miu = 0.03
X = [1.00000000e-03,4.64970350e+00,7.00000000e-01,8.53058723e-03,1.00133269e+01,5.00000000e+01,1.50130392e+01,1.00293814e-01,8.76291319e-02]
lossess20 = X[0]
Nintmax = X[1]
Nintcrit = X[2]
dNextoutdt = X[3]
Ks = X[4]
Vmax = X[5]
KI = X[6]
K0 = X[7]
Ka = X[8]

In [8]:
# optimized parameters: bounds
problem = {
    'num_vars': 6,
    'names': ['Smin','Sopt','Smax','Topt','Tmax','n'],
    'bounds': [[0,10],
               [15,35],
               [40,50],
               [15, 25],
               [31,37],
               [1,6]]
}

print(problem)

{'num_vars': 6, 'names': ['Smin', 'Sopt', 'Smax', 'Topt', 'Tmax', 'n'], 'bounds': [[0, 10], [15, 35], [40, 50], [15, 25], [31, 37], [1, 6]]}


In [9]:
# number of examined values per parameter - 10
param_values = saltelli.sample(problem, 500)
print(param_values.shape)

(7000, 6)


In [ ]:
# optimization

Y1 = np.zeros([param_values.shape[0]])
Y2 = np.zeros([param_values.shape[0]])

evaluate_model1 = []
evaluate_model2 = []


returns = [1, 2]

w = 0

for p, X in enumerate(param_values):

    Smin = X[0]
    Sopt = X[1]
    Smax = X[2]
    Topt = X[3]
    Tmax = X[4]
    n = X[5]


    # growth and N dynamics figures (based on IMS data) 
    DATA = []
    Errors, RMSRE_m = [], []
    Errors, RMSRE_Nint = [], []
    RMSRE_m_all, RMSRE_Nint_all = [],[]
    returns = [1, 2]

    # model simulation
    for k in returns:
        Square_Errors_m = []
        Square_Errors_Nint = []

        t0 = t0_all[k-1]
        t = list(range(t0,tf_all[k-1]))

        # T Data 
        TV_reduced = TV_average[k-1][0:len(t)]
        f1 = interpolate.interp1d(t, TV_reduced,kind = 'linear')

        # I Data
        IV_reduced = IV_average[k-1][0:len(t)]
        f0 = interpolate.interp1d(t, IV_reduced,kind = 'linear')

        xlabels = xlabels_all[k-1]

        for j in range(len(mi_all[k-1])):
            #print('j = ' + str(j))

            # calculate and plot model results for each initial condition (runs only on odd numbers, to evoid dubble work)
            if j % 2 == 0:
                NEXT, NINT, M, T = [],[],[],[]

                line = '--'
                S = S_all[k-1][j]
                m0 = 0.15 * mi_all[k-1][j]
                Nint0 = Nint0_all[k-1][j]
                d = d_all[k-1][j]
                Next0 = Next00 / (1 + d)
                x0 = n_reactors*[Next0, Nint0, m0]

                # setup initial conditions identical for all reactors
                args = (Nintcrit,Nintmax,Nintmin,Vmax,Ks,KN,miu,S,Z,KI,K0,Ka,Topt,Tmin,Tmax,losses20,
                        teta,Sopt,Smin,Smax,f1,f0,dilution,n,umol_to_percent_DW,dNextoutdt)

                # solve the ODEs using the new syntax
                sol = solve_ivp(Bottles_IMS, [t[0], t[-1]], x0, args=args, t_eval = t)

                # take the solution of the state variables:
                NEXT.append(sol.y[0,:]/1000*62)
                NINT.append(sol.y[1,:])
                M.append(sol.y[2,:])

                T.append(sol.t)

                t = T[0]

            data = {'T':T[0],'Next':NEXT[0],'Nint':NINT[0],'m':M[0]}
            DATA.append(data)

            # m error

            final_m = round(M[-1][-1],2)
            measured_m = round(mf_real[k-1][j],2)
            error_m = round((measured_m - final_m/0.15) / measured_m,3)
            square_error_m = error_m**2
            Square_Errors_m.append(square_error_m)
            RMSRE_m_all.append((square_error_m)) # Square_Errors_m and RMSRE_m_all are the same in the mean while

            # Nint error

            final_Nint = round(NINT [-1][-1],2)

            if Nintf_exps[k-1][j] == j + 2: # ignoring samples without Nintf measurement
                measured_Nint = round(Nintf_real[k-1][j],2)
                error_Nint = round((measured_Nint - final_Nint) / measured_Nint,3)
                square_error_Nint = error_Nint**2
                Square_Errors_Nint.append(square_error_Nint)
                RMSRE_Nint_all.append((square_error_Nint))

            plt.tight_layout()

        Square_Errors_m,Square_Errors_Nint = [],[]

        ...
        ...

    RMSREm = (np.mean(RMSRE_m_all))**0.5
    RMSRENint = (np.mean(RMSRE_Nint_all))**0.5
    evaluate_model1.append(RMSREm)
    evaluate_model2.append(RMSRENint)

    Y1[p] = round(evaluate_model1[-1],3)
    Y2[p] = round(evaluate_model2[-1],3)

    print(w)
    w = w+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# sensitivity analysis (Sobol method) of the model to the different parameters

fig,ST2 = plt.subplots(1,1,sharex=True,figsize=(4,5))
ST2.plot(Si2['ST'],problem['names'],'*',markersize=7,color='dodgerblue')
ST2.plot(Si1['ST'],problem['names'],'.',markersize=10,color='black')


ST2.set_xlabel('Sensitivity Index',fontsize=14, weight="bold")
ST2.set_ylabel('Parameter',fontsize=14, weight="bold")
ST2.set_xlim([0, 1])
ST2.set_axisbelow(True)
ST2.yaxis.grid(color='lightgray', linestyle='dashed')
ST2.legend(['Internal N','m'])


name = 'Brine Sensitivity.png' 
fig.savefig(fname=name, dpi=600)

In [ ]:
Y3 = Y1 + Y2
from sklearn.linear_model import LinearRegression
Best_index = []
for i in range(len(Y1)):
    if (Y1[i] < 5) & (Y2[i] < 5):
        Best_index.append(i)
        
b = param_values[Best_index].T

fig,ax0 = plt.subplots(2,3,figsize=(18,21),sharey=True)#,sharex=True)

for j in range(len(b)):
    ax0.flat[j].plot(b[j],Y1[Best_index],'.',markersize=4,color = 'lightblue')
    ax0.flat[j].plot(b[j],Y2[Best_index],'.',markersize=4,color = 'pink')
    ax0.flat[j].plot(b[j],Y3[Best_index],'.',markersize=6,color = 'lightgreen')
    ax0.flat[j].set_xlabel(str(problem['names'][j]),fontsize=11, weight="bold")
    
    
    # regression line and coefficient:
    mm, bb = np.polyfit(b[j], Y1[Best_index], 1)
    ax0.flat[j].plot(b[j], mm*b[j] + bb,'.',color = 'blue',markersize=4)
    x = np.array(b[j]).reshape((-1, 1))
    model = LinearRegression()
    model.fit(x, Y1[Best_index])
    model = LinearRegression().fit(x, Y1[Best_index])
    r_sq = model.score(x, Y1[Best_index])

    # regression line and coefficient for Nint:
    NN, cc = np.polyfit(b[j], Y2[Best_index], 1)
    ax0.flat[j].plot(b[j], NN*b[j] + cc,'.',color = 'red',markersize=4)
    x = np.array(b[j]).reshape((-1, 1))
    modelN = LinearRegression()
    modelN.fit(x, Y2[Best_index])
    modelN = LinearRegression().fit(x, Y2[Best_index])
    r_sqN = modelN.score(x, Y2[Best_index])

    
    # Combined (Y3)
    
    #ax0.flat[j].plot(b[j],Y2[Best_index],'.',markersize=6,color = 'pink')
    #ax0.flat[j].set_xlabel(str(problem['names'][j]),fontsize=11, weight="bold")
    #ax0.flat[j].set_ylabel('RMSRE',fontsize=12, weight="bold")
    #ax0.flat[j].set_xlim(problem['bounds'][0])
    
    # regression line and coefficient:
    MN, DD = np.polyfit(b[j], Y3[Best_index], 1)
    ax0.flat[j].plot(b[j], MN*b[j] + DD,'.',color = 'darkgreen',markersize=4)
    x = np.array(b[j]).reshape((-1, 1))
    model = LinearRegression()
    model.fit(x, Y3[Best_index])
    model = LinearRegression().fit(x, Y3[Best_index])
    r_sqC = model.score(x, Y3[Best_index])
    
    ax0.flat[j].legend([['$R^{2}$ = ' + str(round(r_sq,2))],['$R^{2}$ = ' + str(round(r_sqN,2))],['$R^{2}$ = ' + str(round(r_sqC,2))]],fontsize = 14,loc = 'upper left',markerscale = 2.5)

    
plt.tight_layout()
plt.show()

name = 'Brine Full Error plots.png' 
    
fig.savefig(fname=name, dpi=600)

In [ ]:
# parameters achieving lowest errors for m, Nint and combined

minerrorm = min(Y1)
minerrorN = min(Y2)
minsumerrors = min(Y3)

merror_best = Y1[Y3.tolist().index(min(Y3))]
Nerror_best = Y2[Y3.tolist().index(min(Y3))]

indminerrorm = Y1.tolist().index(min(Y1))
indminerrorN = Y2.tolist().index(min(Y2))
BestParam_m = param_values[Y1.tolist().index(min(Y1))]
BestParam_N = param_values[Y2.tolist().index(min(Y2))]
BestParam = param_values[Y3.tolist().index(min(Y3))]

print('Minimum error for m: ' + str(minerrorm))
print('Minimum error for Nint: ' + str(minerrorN) + '\n')
print('Minimum sum of errors: ' + str(minsumerrors) + '\n')
print('Best error for m: ' + str(merror_best))
print('Best error for N: ' + str(Nerror_best))

for i in range(len(problem['names'])):
    print('Parameter: ' + str(problem['names'][i]) + ' Best values: ' + str(round(BestParam_m[i],3)) + ' , ' +str(round(BestParam_N[i],3)) + ' and: ' +str(round(BestParam[i],3))+'\n')

In [ ]:
# parameters for combined lowest error
X = param_values[Y3.tolist().index(min(Y3))].tolist()
X

In [ ]:
#growth and N dynamics figures (based on IMS data) 

DATA = []
Errors, RMSRE_m = [], []
Errors, RMSRE_Nint = [], []
RMSRE_m_all, RMSRE_Nint_all = [],[]
returns = [1, 2]

# model simulation
for k in returns:
    Square_Errors_m = []
    Square_Errors_Nint = []
    
    print('Run #' + str(k) + '\n')
    
    fig, ax = plt.subplots(3,2,figsize=(13,8))
    fig2, ax2 = plt.subplots(2,1,figsize=(13,6))
    
    t0 = t0_all[k-1]
    t = list(range(t0,tf_all[k-1]))
    
    # T Data 
    TV_reduced = TV_average[k-1][0:len(t)]
    f1 = interpolate.interp1d(t, TV_reduced,kind = 'linear')
    
    # I Data
    IV_reduced = IV_average[k-1][0:len(t)]
    f0 = interpolate.interp1d(t, IV_reduced,kind = 'linear')
    
    xlabels = xlabels_all[k-1]
    
    for j in range(len(mi_all[k-1])):
        print('j = ' + str(j))
        
        # calculate and plot model results for each initial condition (runs only on odd numbers, to evoid dubble work)
        if j % 2 == 0:
            NEXT, NINT, M, T = [],[],[],[]

            line = '--'
            S = S_all[k-1][j]
            m0 = 0.15 * mi_all[k-1][j]
            Nint0 = Nint0_all[k-1][j]
            d = d_all[k-1][j]
            Next0 = Next00 / (1 + d)
            x0 = n_reactors*[Next0, Nint0, m0]
            
            if d == 0 and S < 10:
                color = 'blue'
            elif d == max(d_all[k-1]):
                color = 'turquoise'
            elif d != 0 and d != max(d_all[k-1]):
                color = 'dodgerblue'
            elif d == 0 and S > 10:
                color = 'red'
                line = '-.'
                
            # setup initial conditions identical for all reactors
            args = (Nintcrit,Nintmax,Nintmin,Vmax,Ks,KN,miu,S,Z,KI,K0,Ka,Topt,Tmin,Tmax,losses20,
                    teta,Sopt,Smin,Smax,f1,f0,dilution,n,umol_to_percent_DW,dNextoutdt)

            # solve the ODEs using the new syntax
            sol = solve_ivp(Bottles_IMS, [t[0], t[-1]], x0, args=args, t_eval = t)

            # take the solution of the state variables:
            NEXT.append(sol.y[0,:]/1000*62)
            NINT.append(sol.y[1,:])
            M.append(sol.y[2,:])

            T.append(sol.t)

            t = T[0]
            if mi_all[k-1][j] < max(mi_all[k-1]):
                r = 0
            else:
                r = 1
            l = l_all[k-1][r]

            ax[0,r].plot(t,NEXT[-1],linestyle = line,markersize=3.5,color=color)
            ax[0,r].set_ylim([0,Next00*1.2/1000*62])
            ax[1,r].plot(t,NINT[-1],linestyle = line,markersize=3.5,color=color)
            ax[1,r].set_ylim([0.5,5.5])
            ax[2,r].plot(t,M[-1]/0.15,linestyle = line,markersize=3.5,color=color)
            
            col = len(l)
            ax[0,r].legend(l,ncol=col,loc='upper right',fontsize='medium',markerscale=2,columnspacing = 0.2,handletextpad = 0.3)#,edgecolor='white')
        
        data = {'T':T[0],'Next':NEXT[0],'Nint':NINT[0],'m':M[0]}
        DATA.append(data)
        ax[2,r].plot(t[-1],mf_real[k-1][j],'*',markersize = 6,color=color)
        
        # Mark real Nint data as a star
        
        
        print('exp: ' + str(j+2))
        print('Real Nint is: ' + str(Nintf_exps[k-1][j]))
        if Nintf_exps[k-1][j] == j + 2:
            print('exp: ' + str(j + 2))
            ax[1,r].plot(t[-1],Nintf_real[k-1][j],'*',markersize = 6,color=color)
                
        ax[0,r].set_xticklabels([])
        ax[1,r].set_xticklabels([])
        ax[2,r].set_xticks(t[0:len(t):int(len(t)/2-1)])
        ax[2,r].set_xticklabels([str(i) for i in xlabels], rotation=45,fontsize=10, weight="bold")

        ax[0,r].set_ylabel('Next \n [ppm $NO3^{-}$]', fontsize=12, weight="bold")
        ax[1,r].set_ylabel('Nint \n [% g N $g DW^{-1}$]',fontsize=12, weight="bold")
        ax[2,r].set_ylabel('m \n [g FW $l^{-1}$]',fontsize=12, weight="bold")
        
        plt.tight_layout()

        # plot I and T
        ax2[0].plot(t,TV_reduced,'.',markersize=3,color='black')
        ax2[1].plot(t,IV_reduced,'.',markersize=3,color='k')
            
        ax2[0].set_ylabel('Temperature \n[C]',fontsize=11, weight="bold")
        ax2[1].set_ylabel('Light Intensity \n[µmol photons $m^{-2}$ $second^{-1}$]',fontsize=11, weight="bold")

        ax2[0].set_xticklabels([])
        ax2[1].set_xticklabels([])
        ax2[1].set_xticks(t[0:len(t):int(len(t)/2-1)])
        ax2[1].set_xticklabels([str(i) for i in xlabels], rotation=45,fontsize=10, weight="bold")
        
        # calculate errors
        
        # m error
        
        final_m = round(M[-1][-1],2)
        #print(final_m)
        measured_m = round(mf_real[k-1][j],2)
        #print(measured_m)
        error_m = round((measured_m - final_m/0.15) / measured_m,3)
        square_error_m = error_m**2
        #print(square_error_m)
        
        Square_Errors_m.append(square_error_m)
        RMSRE_m_all.append((square_error_m)) # Square_Errors_m and RMSRE_m_all are the same in the mean while
        
        print('The modeled m is: ' + str(round(final_m/0.15,2)) + '[g FW/bottle], The measured m is ' + str(measured_m) + '[g FW/bottle]')
        print('The calculated error is: ' + str(error_m) + '\n')
        
        # Nint error
        
        final_Nint = round(NINT [-1][-1],2)
        
        if Nintf_exps[k-1][j] == j + 2: # ignoring samples without Nintf measurement
            measured_Nint = round(Nintf_real[k-1][j],2)
            error_Nint = round((measured_Nint - final_Nint) / measured_Nint,3)
            square_error_Nint = error_Nint**2
            #print(square_error_Nint)
            Square_Errors_Nint.append(square_error_Nint)
            RMSRE_Nint_all.append((square_error_Nint))
            
            print('The modeled Nint is: ' + str(round(final_Nint,2)) + '[% gN /g DW], The measured Nint is ' + str(measured_Nint) + '[% gN /g DW]')
            print('The calculated error is: ' + str(error_Nint) + '\n')
            
        plt.tight_layout()
    
    name = 'fig' + str(k) + 'a.png' 
    name2 = 'fig' + str(k) + 'b.png' 
    fig.savefig(fname=name, dpi=600)
    fig2.savefig(fname=name2, dpi=600)
    
    RMSRE_m.append((np.mean(Square_Errors_m))**0.5)
    

    #RMSRE_m_all.append((Square_Errors_m))
    print('for return ' + str(k) + ', RMSRE of m = ' + str(round(RMSRE_m[-1],2)) + '\n')
    
    RMSRE_Nint.append((np.mean(Square_Errors_Nint))**0.5)
    
    #print('for return ' + str(k) + ', RMSRE of Nint = ' + str(round(RMSRE_Nint[-1],2)) + '\n')

    Square_Errors_m,Square_Errors_Nint = [],[]

RMSRE_m_tot = (np.mean(RMSRE_m_all))**0.5
RMSRE_Nint_tot = (np.mean(RMSRE_Nint_all))**0.5
#print(RMSREm)
#print(RMSRENint)

print('The total errors are: RMSRE of m = ' + str(round(RMSRE_m_tot,3)) + ' RMSRE of Nint = ' + str(round(RMSRE_Nint_tot,3)) + '\n')


In [ ]:
print('The total errors are: RMSRE of m = ' + str(round(RMSRE_m_tot,3)) + ' RMSRE of Nint = ' + str(round(RMSRE_Nint_tot,3)) + '\n')